In [ ]:
###############################################################################################################################



###############################################################################################################################

# Analysis of Classification Efficiency Yelp data using frequency bag-of-words

In [4]:
import re
import pandas as pd
import numpy as np

In [5]:
def TextPreprocess(text):   
    text = text.str.lower()
    text = text.str.replace('[^\w\s]','')    
    text = text.str.replace('[0-9]','')
    text = text.str.replace('_','')
    return text

In [6]:
vocab = pd.read_csv('yelp-vocab.txt',header=None,names = ['word','id','frequency'])
#vocabtotext = vocab.word.str.cat(sep=' ')

In [7]:
loc = 'yelp-train.txt'
yelp_train = pd.read_table(loc,header=None,names=['review','label'])
yelp_train.review = TextPreprocess(yelp_train.review)

In [8]:
loc = 'yelp-test.txt'
yelp_test = pd.read_table(loc,header=None,names=['review','label'])
yelp_test.review = TextPreprocess(yelp_test.review)

In [9]:
loc = 'yelp-valid.txt'
yelp_valid = pd.read_table(loc,header=None,names=['review','label'])
yelp_valid.review = TextPreprocess(yelp_valid.review)

In [ ]:
#############################################################################################################################
'''
One sprase matrix constructed save it and call the data when its needed as computation is time consuming
'''

############################################################################################################################

In [ ]:
##########################################################################################################################



##########################################################################################################################

In [2]:
import scipy.sparse
train_mat = scipy.sparse.load_npz('yelp_train_freq_mat.npz')
test_mat = scipy.sparse.load_npz('yelp_test_freq_mat.npz')
valid_mat = scipy.sparse.load_npz('yelp_valid_freq_mat.npz')


In [ ]:
#train_set = yelp_train.iloc[np.random.choice(len(yelp_train.review),20)]       # randomly picking dataset to train
#validation_set = yelp_valid.iloc[np.random.choice(len(yelp_valid.review),20)]      # randomly picking dataset to classify

In [10]:
train_y_true = yelp_train.label
valid_y_true = yelp_valid.label
test_y_true = yelp_test.label

### Applying Naive bayes classifier

In [11]:
# computeing accuracy and F1 score for predicting "training data"
# input sparse matrix data that needs to be predicted
def getClassifierEff (Data,true_y,clf):
    from sklearn.metrics import f1_score,accuracy_score
    y_pred = clf.predict(Data)
    acc = accuracy_score(true_y,y_pred)
    f1 = f1_score(true_y,y_pred,average='micro')
    return f1,acc

In [25]:
# training classifier with "training data"
from sklearn.naive_bayes import MultinomialNB

In [54]:
Hyp = [0, 1e-10, 1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1,1]
yelp_valid_f1 = []
yelp_valid_acc = []
for itr in range(len(Hyp)):
    naive_clf = MultinomialNB(alpha=Hyp[itr]).fit(train_mat, train_y_true)
    f1 , acc = getClassifierEff (valid_mat,valid_y_true,naive_clf)
    yelp_valid_f1.append(f1)
    yelp_valid_acc.append(acc)
   # print('yelp_valid_f1',yelp_valid_f1)

d:\machine learning\python 3.5.2\lib\site-packages\sklearn\naive_bayes.py:472: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


In [55]:
yelp_valid_f1

[0.439,
 0.439,
 0.44,
 0.444,
 0.446,
 0.449,
 0.458,
 0.469,
 0.479,
 0.491,
 0.507,
 0.516]

In [56]:
optim_alpha = Hyp[np.argmax(yelp_valid_f1)]
print('for hyper parameter alpha =',Hyp[np.argmax(yelp_valid_f1)],'we get max F1 score')

for hyper parameter alpha = 1 we get max F1 score


In [57]:
naive_clf = BernoulliNB( alpha=optim_alpha ).fit(train_mat, train_y_true)

train_f1 , train_acc = getClassifierEff (train_mat,train_y_true,naive_clf)
test_f1 , test_acc = getClassifierEff (test_mat,test_y_true,naive_clf)

print('yelp_train_f1',train_f1)
print('yelp_test_f1',test_f1)


yelp_train_f1 0.5974285714285714
yelp_test_f1 0.4085


In [30]:
from sklearn.naive_bayes import GaussianNB

In [33]:
yelp_valid_f1 = []
yelp_valid_acc = []

naive_clf = GaussianNB().fit(train_mat.toarray(), train_y_true)
f1 , acc = getClassifierEff (valid_mat.toarray(),valid_y_true,naive_clf)
yelp_valid_f1 = f1
yelp_valid_acc = acc

train_f1 , train_acc = getClassifierEff (train_mat.toarray(),train_y_true,naive_clf)
test_f1 , test_acc = getClassifierEff (test_mat.toarray(),test_y_true,naive_clf)

MemoryError: 

### Applying Decision Tree

In [17]:
from sklearn import tree

class sklearn.tree.DecisionTreeClassifier(criterion=’gini’, splitter=’best’, max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, class_weight=None, presort=False)

In [18]:
crit = ['gini','entropy'] #criterion 
split = ['best','random'] #splitter


In [19]:
yelp_valid_f1 = []
yelp_valid_acc = []
for itr1 in range(len(crit)):
    for itr2 in range(len(split)):
        tree_clf = tree.DecisionTreeClassifier(criterion = crit[itr1], splitter = split[itr2]).fit(train_mat, train_y_true)
        f1,acc = getClassifierEff (valid_mat,valid_y_true,tree_clf)
        yelp_valid_f1.append(f1)
        yelp_valid_acc.append(acc)
        
        print('yelp_valid_f1',f1,'when we use criterion',crit[itr1],'and splitter',split[itr2])

yelp_valid_f1 0.35 when we use criterion gini and splitter best
yelp_valid_f1 0.361 when we use criterion gini and splitter random
yelp_valid_f1 0.326 when we use criterion entropy and splitter best
yelp_valid_f1 0.37099999999999994 when we use criterion entropy and splitter random


In [20]:
tree_clf = tree.DecisionTreeClassifier(criterion = 'entropy', splitter = 'best').fit(train_mat, train_y_true)
yelp_train_f1,yelp_train_acc = getClassifierEff (train_mat,train_y_true,tree_clf)
yelp_test_f1,yelp_test_acc = getClassifierEff (test_mat,test_y_true,tree_clf)

print('yelp_train_f1',yelp_train_f1)
print('yelp_test_f1',yelp_test_f1)

yelp_train_f1 1.0
yelp_test_f1 0.3285


### Applying Linear SVC

class sklearn.svm.LinearSVC(penalty=’l2’, loss=’squared_hinge’, dual=True, tol=0.0001, C=1.0, multi_class=’ovr’, fit_intercept=True, intercept_scaling=1, class_weight=None, verbose=0, random_state=None, max_iter=1000)

In [21]:
from sklearn.svm import LinearSVC
linear_clf = LinearSVC().fit(train_mat, train_y_true)

### for combination of penalty='l2' , loss='hinge',dual=True

In [43]:
pen = ['l2']
los = ['hinge'] # gives error for hinge
dul = [True] # gives error for dual = true. 
tolerance = [1e-10, 1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1]
C_param = [1e-10, 1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1]

In [47]:
yelp_valid_f1 = []
yelp_valid_acc = []
for itr1 in range(len(pen)):

    linear_clf = LinearSVC(penalty='l2' , loss='hinge', dual=True).fit(train_mat, train_y_true)
    f1 , acc = getClassifierEff (valid_mat,valid_y_true,linear_clf)
    yelp_valid_f1.append(f1)
    yelp_valid_acc.append(acc)
            
    print('yelp_valid_f1',f1,'when we use penalty',pen[itr1],'loss',los[itr1],'dual',dul[itr1])

yelp_valid_f1 0.44 when we use penalty l2 loss hinge dual True


In [48]:
yelp_valid_f1 = []
yelp_valid_acc = []
for itr1 in range(len(pen)):
    for itr2 in range(len(tolerance)):
        for itr3 in range(len(C_param)):
            linear_clf = LinearSVC(penalty=pen[itr1],tol=tolerance[itr2],C=C_param[itr3],dual=False).fit(train_mat, train_y_true)
            f1 , acc = getClassifierEff (valid_mat,valid_y_true,linear_clf)
            yelp_valid_f1.append(f1)
            yelp_valid_acc.append(acc)
            
            print('yelp_valid_f1',f1,'when we use penalty',pen[itr1],'tolerence',tolerance[itr2],'C',C_param[itr3])

yelp_valid_f1 0.35500000000000004 when we use penalty l2 tolerence 1e-10 C 1e-10
yelp_valid_f1 0.35500000000000004 when we use penalty l2 tolerence 1e-10 C 1e-09
yelp_valid_f1 0.35500000000000004 when we use penalty l2 tolerence 1e-10 C 1e-08
yelp_valid_f1 0.357 when we use penalty l2 tolerence 1e-10 C 1e-07
yelp_valid_f1 0.36499999999999994 when we use penalty l2 tolerence 1e-10 C 1e-06
yelp_valid_f1 0.405 when we use penalty l2 tolerence 1e-10 C 1e-05
yelp_valid_f1 0.461 when we use penalty l2 tolerence 1e-10 C 0.0001
yelp_valid_f1 0.5 when we use penalty l2 tolerence 1e-10 C 0.001
yelp_valid_f1 0.508 when we use penalty l2 tolerence 1e-10 C 0.01
yelp_valid_f1 0.476 when we use penalty l2 tolerence 1e-10 C 0.1
yelp_valid_f1 0.35500000000000004 when we use penalty l2 tolerence 1e-09 C 1e-10
yelp_valid_f1 0.35500000000000004 when we use penalty l2 tolerence 1e-09 C 1e-09
yelp_valid_f1 0.35500000000000004 when we use penalty l2 tolerence 1e-09 C 1e-08
yelp_valid_f1 0.357 when we use pen

In [50]:
linear_clf = LinearSVC(penalty='l2',tol=0.01,C=0.01,loss='hinge',dual=True).fit(train_mat, train_y_true)
yelp_train_f1,yelp_train_acc = getClassifierEff (train_mat,train_y_true,linear_clf)
yelp_test_f1,yelp_test_acc = getClassifierEff (test_mat,test_y_true,linear_clf)
yelp_valid_f1,yelp_valid_acc = getClassifierEff (valid_mat,valid_y_true,linear_clf)
print('yelp_train_f1',yelp_train_f1)
print('yelp_test_f1',yelp_test_f1)
print('yelp_valid_f1',yelp_valid_f1)

yelp_train_f1 0.7611428571428572
yelp_test_f1 0.514
yelp_valid_f1 0.507


### for combination of penalty='l1','l2' , loss='squred_hinge',dual=False

In [51]:
pen = ['l1','l2']
los = ['squared_hinge'] # gives error for hinge
dul = [False] # gives error for dual = true. 
tolerance = [1e-10, 1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1]
C_param = [1e-10, 1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1]

In [52]:
yelp_valid_f1 = []
yelp_valid_acc = []
for itr1 in range(len(pen)):
    for itr2 in range(len(tolerance)):
        for itr3 in range(len(C_param)):
            linear_clf = LinearSVC(penalty=pen[itr1],tol=tolerance[itr2],C=C_param[itr3],dual=False).fit(train_mat, train_y_true)
            f1 , acc = getClassifierEff (valid_mat,valid_y_true,linear_clf)
            yelp_valid_f1.append(f1)
            yelp_valid_acc.append(acc)
            
            print('yelp_valid_f1',f1,'when we use penalty',pen[itr1],'tolerence',tolerance[itr2],'C',C_param[itr3])

yelp_valid_f1 0.084 when we use penalty l1 tolerence 1e-10 C 1e-10
yelp_valid_f1 0.084 when we use penalty l1 tolerence 1e-10 C 1e-09
yelp_valid_f1 0.084 when we use penalty l1 tolerence 1e-10 C 1e-08
yelp_valid_f1 0.084 when we use penalty l1 tolerence 1e-10 C 1e-07
yelp_valid_f1 0.084 when we use penalty l1 tolerence 1e-10 C 1e-06
yelp_valid_f1 0.084 when we use penalty l1 tolerence 1e-10 C 1e-05
yelp_valid_f1 0.348 when we use penalty l1 tolerence 1e-10 C 0.0001
yelp_valid_f1 0.375 when we use penalty l1 tolerence 1e-10 C 0.001
yelp_valid_f1 0.45 when we use penalty l1 tolerence 1e-10 C 0.01
yelp_valid_f1 0.482 when we use penalty l1 tolerence 1e-10 C 0.1
yelp_valid_f1 0.084 when we use penalty l1 tolerence 1e-09 C 1e-10
yelp_valid_f1 0.084 when we use penalty l1 tolerence 1e-09 C 1e-09
yelp_valid_f1 0.084 when we use penalty l1 tolerence 1e-09 C 1e-08
yelp_valid_f1 0.084 when we use penalty l1 tolerence 1e-09 C 1e-07
yelp_valid_f1 0.084 when we use penalty l1 tolerence 1e-09 C 1e-0

yelp_valid_f1 0.357 when we use penalty l2 tolerence 1e-08 C 1e-07
yelp_valid_f1 0.36499999999999994 when we use penalty l2 tolerence 1e-08 C 1e-06
yelp_valid_f1 0.405 when we use penalty l2 tolerence 1e-08 C 1e-05
yelp_valid_f1 0.461 when we use penalty l2 tolerence 1e-08 C 0.0001
yelp_valid_f1 0.5 when we use penalty l2 tolerence 1e-08 C 0.001
yelp_valid_f1 0.508 when we use penalty l2 tolerence 1e-08 C 0.01
yelp_valid_f1 0.476 when we use penalty l2 tolerence 1e-08 C 0.1
yelp_valid_f1 0.35500000000000004 when we use penalty l2 tolerence 1e-07 C 1e-10
yelp_valid_f1 0.35500000000000004 when we use penalty l2 tolerence 1e-07 C 1e-09
yelp_valid_f1 0.35500000000000004 when we use penalty l2 tolerence 1e-07 C 1e-08
yelp_valid_f1 0.357 when we use penalty l2 tolerence 1e-07 C 1e-07
yelp_valid_f1 0.36499999999999994 when we use penalty l2 tolerence 1e-07 C 1e-06
yelp_valid_f1 0.405 when we use penalty l2 tolerence 1e-07 C 1e-05
yelp_valid_f1 0.461 when we use penalty l2 tolerence 1e-07 C 0.

In [53]:
linear_clf = LinearSVC(penalty='l2',tol=0.01,C=0.01,loss='squared_hinge',dual=False).fit(train_mat, train_y_true)
yelp_train_f1,yelp_train_acc = getClassifierEff (train_mat,train_y_true,linear_clf)
yelp_test_f1,yelp_test_acc = getClassifierEff (test_mat,test_y_true,linear_clf)
yelp_valid_f1,yelp_valid_acc = getClassifierEff (valid_mat,valid_y_true,linear_clf)
print('yelp_train_f1',yelp_train_f1)
print('yelp_test_f1',yelp_test_f1)
print('yelp_valid_f1',yelp_valid_f1)

yelp_train_f1 0.855
yelp_test_f1 0.5015
yelp_valid_f1 0.508
